# 평가(Evaluation)

## 정확도(Accuracy)

- 정확도 = 예측결과가 동일한 데이터 건수/전체 예측 데이터 건수

- scikit learn 의 BaseEstimator클래스를 상속받으면
  Customized 형태의 Estimator를 개발자가 생성가능
  
- fit() 메서드는 아무것도 수행하지 않음.
- predict() 메서드는 단순히 피처가 1이면 0 그렇치 않으면 1로 예측

In [1]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    #fit 메서드는 아무것도 학습하지 않음
    def fit(self,X,y=None):
        pass
    #predict()메서드는 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측
    def predict(self,X):
        pred=np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i]=0
            else :
                pred[i]=1
        return pred

In [2]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    df['Fare'].fillna(0,inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [3]:
# MyDummyClassfier로 학습/예측/평가 하기

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 타이타닉 데이터 로딩 및 가공/ 데이터 분리
titanic_df = pd.read_csv('titanic_train.csv')
y_df = titanic_df['Survived']
X_df = titanic_df.drop('Survived',axis=1)
X_df = transform_features(X_df)
X_train,X_test,y_train,y_test=train_test_split(X_df,y_df,test_size=0.2, random_state=0)

# DummyClassfier로 학습/예측/평가
Dummyclf = MyDummyClassifier()
Dummyclf.fit(X_train,y_train)
pred = Dummyclf.predict(X_test)

print('MyDummyClassifier 정확도 :{0:4f}'.format(accuracy_score(y_test,pred)))

MyDummyClassifier 정확도 :0.787709


In [4]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

class MyFakeClassifier(BaseEstimator):
    def fit(self,X,y):
        pass

    #입력값으로 들어오는 x데이터 세트의 크기만큼 모두 0으로 만들어서 반환
    def predict(self,X):
        return np.zeros((len(X),1),dtype=bool)

# 내장 데이터 세트 load_digits()를 이용해 MNIST 데이터 로딩
digits = load_digits()


# 불균형한 레이블 분포도 확인
print('레이블 데이터 세트 크기 : ', digits.data.shape)
print('테스트 세트 레이블 0과 1의 분포도: \n',digits.target.shape)

레이블 데이터 세트 크기 :  (1797, 64)
테스트 세트 레이블 0과 1의 분포도: 
 (1797,)


In [5]:
digits.target == 7

array([False, False, False, ..., False, False, False])

In [6]:
# digits 번호가 7번이면 True, 이를 astype(int) 1로 반환, 7이 아니면 false, 0으로 반환
y = (digits.target==7).astype(int)
X_train,X_test,y_train,y_test=train_test_split(digits.data,y,random_state=11)

In [7]:
# 불균형한 레이블 데이터 분포확인
print('레이블 테스트 세트 크기 : ',y_test.shape)
print('테스트 세트 레이블 0과 1의 분포:\n',pd.Series(y_test).value_counts())
# DummyCalssifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train,y_train)
fpred = fakeclf.predict(X_test)
print('모든 예측을 0으로 한 정확도는 : {:.4f}'.format(accuracy_score(y_test,fpred)))

레이블 테스트 세트 크기 :  (450,)
테스트 세트 레이블 0과 1의 분포:
 0    405
1     45
dtype: int64
모든 예측을 0으로 한 정확도는 : 0.9000


# 오차행렬(confusion matrix) 

- 얼마나 헷갈리고 있는지 보여주는 지표
    - TN(True Negative) : 실제값이 Negative인 값을 Negative로 예측(0,0)
    - FN(False Negative) : Negative로 예측 했는데 실제값은 Positive (1,0)
    - FP(False Positive) : Positive로 예측했는데 실제값은 Negative(0,1)
    - TP(True Positive) : 실제값이 Positive인 값을 Positive 로 예측(1,1)
    
    
- 불균형한 데이터에서는 정확도 보다 정밀도, 재현율이 더 선호됨

- 정확도(Accuracy) : (TN+TP)/(TN+FP+FN+TP)
    
- 정밀도(Precision) : (TP)/(TP+FP) = Positive로 예측한 대상 중에 실제값이 Positive인 값의 비율
  
  = 양성예측도
  
    
- 재현율(Recall) : (TP)/(TP+FN) = 실제값이 Positive인 값중에 예측과 실제값이 Positive인 데이터 비율
  
  = 민감도(Sensitivity) = TPR(True Positive Rate)

In [8]:
# MyFakeClassifier 의 예측결과 fpred를 오차행렬를 이용한 배열로 출력

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,fpred)

array([[405,   0],
       [ 45,   0]], dtype=int64)

## 정밀도(Precision)과 재현율(Recall)

-MyFakeClassfier 의 예측결과로 정밀도와 재현율 측정

- 오차행렬, 정확도, 정밀도, 재현율을 한번에 계산하는 함수 생성
        get_clf_val()함수

In [9]:
from sklearn.metrics import accuracy_score, precision_score,recall_score, confusion_matrix

def get_clf_val(y_test,pred):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    
    print('오차행렬:\n', confusion)
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}'.format(accuracy,precision,recall))

### 원본데이터 로드 후 LogisticRegression() 의 정확도, 정밀도, 재현율 확인

In [10]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#원본 데이터 재로딩, 가공, train, test 분리
titanic_df = pd.read_csv('titanic_train.csv')
y = titanic_df['Survived']
X = titanic_df.drop('Survived',axis = 1)
X = transform_features(X)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=11)

lr = LogisticRegression()
lr.fit(X_train,y_train)
pred = lr.predict(X_test)
get_clf_val(y_test,pred)

오차행렬:
 [[104  14]
 [ 13  48]]
정확도 : 0.8492, 정밀도 : 0.7742, 재현율 : 0.7869


C:\Users\pc\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Precision/Recall Trade-off

- Trade-off : 정밀도, 재현율이 특별히 강조되야 할 경우 임계값(Threshold)을
              조정해 정밀도,재현율 수치를 높일 수 있음
              
              절밀도, 재현율 중 어느 한쪽을 강제로 높이면 다른 하나의 수치는 떨어지기 쉬움
              이를 Trade-off 라고 함

- 일반적인 이진분류 : 임계값을 0.5 = 50% 기준 설정

    - 기준값보다 확률이 크면 Positive 작으면, Negative로 결정

- predict_proba() 메서드 : 학습 완료 된 Classfier 에서 호출 가능
    - test 피처 데이터 세트 : 파라미터로 입력 해주면 테스트 피처 레코드의 개별 클래스 예측 확률 반환

- 반환값  : ndarray m x n ( m : 입력값의 레코드 수, n : 클래스 값 유형)
    - ex) 입력 테스트 데이터 표본개수가 100개, 예측 클래스 값 유형 2개(이진분류) 반환값 = 100x2 ndarray
    - 첫번째 컬0럼 : 0 Negative 확률, 두번째 컬럼 : 1 Positive 확률

In [20]:
pred_proba = lr.predict_proba(X_test)
pred = lr.predict(X_test)
print('pred_proba 결과 shape: ',pred_proba.shape)
print('pred_preba array 에서 앞 3개만 추출 : \n',pred_proba[:3])

# 예측확률 array와 예측 결과값 array를 병합(concatenate)해 확률과 결괏값을 한눈에 확인
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1,1)],axis=1)
print('두개의 class 중 더 큰 확률을 클래스 값으로 예측 \n',pred_proba_result[:3])

pred_proba 결과 shape:  (179, 2)
pred_preba array 에서 앞 3개만 추출 : 
 [[0.46196064 0.53803936]
 [0.87860795 0.12139205]
 [0.87714215 0.12285785]]
두개의 class 중 더 큰 확률을 클래스 값으로 예측 
 [[0.46196064 0.53803936 1.        ]
 [0.87860795 0.12139205 0.        ]
 [0.87714215 0.12285785 0.        ]]
